In [1]:
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [2]:
data = pd.read_csv('data/btc_data.csv', index_col=0)

In [3]:
data_crop = data.dropna()

In [4]:
# Sélectionner les caractéristiques et exclure la dernière ligne
features = data_crop.drop(columns=['progression tomorrow', 'target', 'close', 'high', 'low', 'volumefrom']).iloc[:-1, :]
target = data['target'].iloc[:-1]

In [17]:
len(data_crop)

3825

In [27]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import sys
import os

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [18]:
# 1. Préparation des données

X = features.values
y = target.values

window_size = 1000 # Vous pouvez ajuster cette valeur selon vos besoins

# 2. Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Entraînement du modèle avec validation walk-forward

for i in range(window_size, len(X) - 1):
    # Séparation des données en ensembles d'entraînement et de test
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+1]
    y_test = y[i:i+1]

    # Normalisation des caractéristiques
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entraînement du modèle
    model.fit(X_train, y_train, epochs=10, verbose=0)  # Vous pouvez ajuster le nombre d'epochs selon vos besoins

    # Évaluation du modèle (optionnel)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Step {i}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


Step 3800, Loss: 0.9602, Accuracy: 0.0000
Step 3801, Loss: 0.8894, Accuracy: 0.0000
Step 3802, Loss: 0.6581, Accuracy: 1.0000
Step 3803, Loss: 0.7091, Accuracy: 0.0000
Step 3804, Loss: 0.4818, Accuracy: 1.0000
Step 3805, Loss: 1.4691, Accuracy: 0.0000
Step 3806, Loss: 0.6157, Accuracy: 1.0000
Step 3807, Loss: 0.8107, Accuracy: 0.0000
Step 3808, Loss: 0.6844, Accuracy: 1.0000
Step 3809, Loss: 2.6761, Accuracy: 0.0000
Step 3810, Loss: 0.7100, Accuracy: 0.0000
Step 3811, Loss: 0.4347, Accuracy: 1.0000
Step 3812, Loss: 0.5534, Accuracy: 1.0000
Step 3813, Loss: 0.5056, Accuracy: 1.0000
Step 3814, Loss: 0.9862, Accuracy: 0.0000
Step 3815, Loss: 0.6037, Accuracy: 1.0000
Step 3816, Loss: 0.2259, Accuracy: 1.0000
Step 3817, Loss: 1.4225, Accuracy: 0.0000
Step 3818, Loss: 0.4067, Accuracy: 1.0000
Step 3819, Loss: 0.3745, Accuracy: 1.0000
Step 3820, Loss: 0.2248, Accuracy: 1.0000
Step 3821, Loss: 1.6753, Accuracy: 0.0000
Step 3822, Loss: 1.1853, Accuracy: 0.0000


In [28]:
# 1. Définir la métrique personnalisée

def earn_metric(predicted_probs, progressions, n_days):
    base = c = 1
    for j in range(n_days):
        index = len(predicted_probs) - n_days + j
        c *= predicted_probs[index] * progressions[index] + (1 - predicted_probs[index])
        base *= progressions[index]
    return c / base

# Préparation des données

X = features.values
y = target.values

window_size = 2000  
n_days = 30 

# Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle avec validation walk-forward

predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=20, verbose=0)

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)
        #print(f"Step {i}, Earn Metric: {metric_value:.4f}")

print(f"Average Earn Metric: {np.mean(metrics):.4f}")

Average Earn Metric: 0.9799
